In [1]:
ENV["GKS_ENCODING"] = "utf-8"
using Plots
include("/home/julian/Hamburg/LadderDGA/run.jl");

using 8 workers.
      From worker 8:	 Activating environment at `~/Hamburg/LadderDGA/Project.toml`
      From worker 2:	 Activating environment at `~/Hamburg/LadderDGA/Project.toml`
      From worker 4:	 Activating environment at `~/Hamburg/LadderDGA/Project.toml`
      From worker 5:	 Activating environment at `~/Hamburg/LadderDGA/Project.toml`
      From worker 7:	 Activating environment at `~/Hamburg/LadderDGA/Project.toml`
      From worker 3:	 Activating environment at `~/Hamburg/LadderDGA/Project.toml`
      From worker 6:	 Activating environment at `~/Hamburg/LadderDGA/Project.toml`


 Activating environment at `~/Hamburg/LadderDGA/Project.toml`
┌ Warning: ArgParse not working with jupyter notebooks
└ @ Main.LadderDGA /home/julian/Hamburg/LadderDGA/src/DepsInit.jl:47
┌ Warning: Check U and Beta consistency in chi_dir/gamma_dir
└ @ Main.LadderDGA /home/julian/Hamburg/LadderDGA/src/DepsInit.jl:48
┌ Info: Reading Inputs...
└ @ Main.LadderDGA /home/julian/Hamburg/LadderDGA/src/DepsInit.jl:50
┌ Info: Reading Fortran Input, this can take several minutes.
└ @ Main.LadderDGA /home/julian/Hamburg/LadderDGA/src/IO.jl:94
┌ Info: Done Reading Gamma
└ @ Main.LadderDGA /home/julian/Hamburg/LadderDGA/src/IO.jl:108
┌ Info: Done Reading chi
└ @ Main.LadderDGA /home/julian/Hamburg/LadderDGA/src/IO.jl:110
┌ Info: loading from 
│   env.inputVars = vars.jld
└ @ Main.LadderDGA /home/julian/Hamburg/LadderDGA/src/helpers.jl:66
┌ Warning: TODO: check beta consistency, config <-> g0man, chi_dir <-> gamma dir
└ @ Main.LadderDGA /home/julian/Hamburg/LadderDGA/src/helpers.jl:76
┌ Info: Inputs R

[ Info: Done.


In [86]:
sP = LadderDGA.simParams
mP = LadderDGA.modelParams
iνₙ = LadderDGA.iν_array(mP.β, 0:(sP.n_iν-1))
ϵqGrid = LadderDGA.reduce_kGrid(LadderDGA.cut_mirror(collect(LadderDGA.ϵkGrid)))
iνₙ, GImp    = LadderDGA.readGImp(LadderDGA.env.inputDir * "/gm_wim", only_positive=true)
ϵₖ, Vₖ, μ    = LadderDGA.read_anderson_parameters(LadderDGA.env.inputDir * "/hubb.andpar");
E_kin_ED, E_pot_ED  = LadderDGA.calc_E_ED(iνₙ[1:41], ϵₖ, Vₖ, GImp[1:41], mP)
E_kin_ED_2, E_pot_ED_2  = LadderDGA.calc_E_ED(iνₙ[1:200], ϵₖ, Vₖ, GImp[1:200], mP)
E_kin_ED_3, E_pot_ED_3  = LadderDGA.calc_E_ED(iνₙ[1:1000], ϵₖ, Vₖ, GImp[1:1000], mP)
E_kin_ED_4, E_pot_ED_4  = LadderDGA.calc_E_ED(iνₙ[1:5000], ϵₖ, Vₖ, GImp[1:5000], mP)
#ekin_ED_l = [E_Kin_ED(iνₙ[1:i], ϵₖ, Vₖ, GImp[1:i], mP.β; full=false) for i in 1:41];
#ekin_ED_l_2 = [E_Kin_ED_2(iνₙ[1:i], Vₖ, GImp[1:i], LadderDGA.Σ_loc_pos, mP.μ, mP.β; full=false) for i in 1:41];
#ekin_ED_l_3 = [2*sum((GImp .* Δ)[1:i]).* (2/mP.β) for i in 1:41];
#epot_ED_l = [E_Pot_ED(iνₙ[1:i], ϵₖ, Vₖ, GImp[1:i], mP) for i in 1:41];

In [3]:
ep, ek = LadderDGA.calc_E(LadderDGA.Σ_loc_pos[1: LadderDGA.simParams.n_iν], [1], 1, (1), LadderDGA.modelParams, LadderDGA.simParams);

TODO: E_pot function has to be tested
TODO: use GNew/GLoc/GImp instead of Sigma
TODO: make frequency summation with sum_freq an optional


In [90]:
function E_ED_fit(iνₙ, ϵₖ, Vₖ, GImp, mP, N)
    NMax = minimum([length(GImp), N]) 
    ffit = Array{Float64,2}(undef, NMax, 2)
    W = LadderDGA.build_weights(1, NMax, [0,6,8])
    for i in 1:NMax
        ffit[i,:] .= LadderDGA.calc_E_ED(iνₙ[1:i], ϵₖ, Vₖ, GImp[1:i], mP)
    end
    E_Kin = LadderDGA.fit_νsum(W, ffit[:,1])
    E_Pot = LadderDGA.fit_νsum(W, ffit[:,2])
    return E_Kin, E_Pot
end

E_ED_fit (generic function with 1 method)

In [92]:
E_ED_fit(iνₙ, ϵₖ, Vₖ, GImp, mP, 200)

(-0.16945059999510464, 0.10263781384649799)

In [10]:
println(E_kin_ED_3)

-0.1691548170827879


In [87]:
println(E_pot_ED)
println(E_kin_ED)
println(E_pot_ED_2)
println(E_kin_ED_2)
println(E_pot_ED_3)
println(E_kin_ED_3)
println(E_pot_ED_4)
println(E_kin_ED_4)

0.10325433395684014
-0.16917397121428923
0.10329915075518104
-0.16915498153769692
0.103299539517137
-0.1691548170827879
0.10329954261168517
-0.1691548157668945


In [291]:
ep, ek = LadderDGA.calc_E(Σ_ladder, ϵqGrid, LadderDGA.qMultiplicity, LadderDGA.modelParams, LadderDGA.simParams);
ep_corr = LadderDGA.E_Pot(Σ_ladder, ϵqGrid, LadderDGA.qMultiplicity, LadderDGA.modelParams, LadderDGA.simParams);
ek_corr = LadderDGA.E_Kin(Σ_ladder, ϵqGrid, LadderDGA.qMultiplicity, LadderDGA.modelParams, LadderDGA.simParams);

┌ Warning: ArgParse not working with jupyter notebooks
└ @ Main.LadderDGA ~/Hamburg/LadderDGA/src/DepsInit.jl:47
┌ Warning: Check U and Beta consistency in chi_dir/gamma_dir
└ @ Main.LadderDGA ~/Hamburg/LadderDGA/src/DepsInit.jl:48
[ Info: Reading Inputs...
[ Info: Reading Fortran Input, this can take several minutes.
[ Info: Done Reading Gamma
[ Info: Done Reading chi
┌ Info: loading from 
└   env.inputVars = "vars.jld"
┌ Warning: TODO: check beta consistency, config <-> g0man, chi_dir <-> gamma dir
└ @ Main.LadderDGA ~/Hamburg/LadderDGA/src/helpers.jl:76
┌ Info: Inputs Read. Starting Computation.
│ Found usable intervals for local susceptibility of length 
│       sp: 21
│       ch: 17 
└       χLoc_sp = 0.4438, χLoc_ch = 0.0512
┌ Warning: Forcing non tail corrected summation in EoM
└ @ Main.LadderDGA ~/Hamburg/LadderDGA/src/DepsInit.jl:100


TODO: E_pot function has to be tested
TODO: use GNew/GLoc/GImp instead of Sigma
TODO: make frequency summation with sum_freq an optional
TODO: E_pot function has to be tested
TODO: use GNew/GLoc/GImp instead of Sigma
TODO: make frequency summation with sum_freq an optional
TODO: E_kin function has to be tested
TODO: use GNew/GLoc/GImp instead of Sigma
TODO: make frequency summation with sum_freq an optional


In [294]:
LadderDGA.writeFortranEnergies(ek, ep, mP.β, ".")

41


In [293]:
pwd()

"/home/julian/Hamburg/LadderDGA"